In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Dropout
from keras.layers import Conv2D,LSTM, BatchNormalization,MaxPooling2D,Reshape
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Loading Data

In [2]:
def load_data():
    X_test = np.load("data/X_test.npy")
    y_test = np.load("data/y_test.npy")
    person_train_valid = np.load("data/person_train_valid.npy")
    X_train_valid = np.load("data/X_train_valid.npy")
    y_train_valid = np.load("data/y_train_valid.npy")
    person_test = np.load("data/person_test.npy")

    return X_test,y_test,person_train_valid,X_train_valid,y_train_valid,person_test

# Prep and Preprocessing

In [3]:
def data_prep(X,y,sub_sample,average,noise):
    
    total_X = None
    total_y = None
    
    X = X[:,:,0:500]
    
    # Maxpooling the data 
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    
    total_X = X_max
    total_y = y
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    
    # Subsampling
    
    for i in range(sub_sample):
        
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
            
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))
    
    return total_X,total_y



In [4]:
def preprocessing(y_train_valid, y_test, X_train_valid, X_test):
    y_train_valid -= 769
    y_test -= 769

    X_train_valid_prep,y_train_valid_prep = data_prep(X_train_valid,y_train_valid,2,2,True)
    X_test_prep,y_test_prep = data_prep(X_test,y_test,2,2,True)

    ## Random splitting and reshaping the data
    total_size = y_train_valid_prep.shape[0]
    num_samples = int(total_size*0.1773)

    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(total_size, num_samples, replace=False)
    ind_train = np.array(list(set(range(total_size)).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid] 
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]


    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test_prep, 4)

    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)


    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)
    
    return y_train, y_valid, y_test, x_train, x_valid, x_test

In [5]:
def generate_basic_cnn_model(): 
    basic_cnn_model = Sequential()

    # Conv. block 1
    basic_cnn_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu', input_shape=(250,1,22)))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same')) # Read the keras documentation
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Conv. block 2
    basic_cnn_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Conv. block 3
    basic_cnn_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Conv. block 4
    basic_cnn_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Output layer with Softmax activation
    basic_cnn_model.add(Flatten()) # Flattens the input
    basic_cnn_model.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation

    return basic_cnn_model


In [6]:
def generate_cnn_lstm_hybrid_model():
    hybrid_cnn_lstm_model = Sequential()

    # Conv. block 1
    hybrid_cnn_lstm_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu', input_shape=(250,1,22)))
    hybrid_cnn_lstm_model.add(MaxPooling2D(pool_size=(3,1), padding='same')) # Read the keras documentation
    hybrid_cnn_lstm_model.add(BatchNormalization())
    hybrid_cnn_lstm_model.add(Dropout(0.5))

    # Conv. block 2
    hybrid_cnn_lstm_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
    hybrid_cnn_lstm_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    hybrid_cnn_lstm_model.add(BatchNormalization())
    hybrid_cnn_lstm_model.add(Dropout(0.5))

    # Conv. block 3
    hybrid_cnn_lstm_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
    hybrid_cnn_lstm_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    hybrid_cnn_lstm_model.add(BatchNormalization())
    hybrid_cnn_lstm_model.add(Dropout(0.5))

    # FC+LSTM layers
    hybrid_cnn_lstm_model.add(Flatten()) # Adding a flattening operation to the output of CNN block
    hybrid_cnn_lstm_model.add(Dense((100))) # FC layer with 100 units
    hybrid_cnn_lstm_model.add(Reshape((100,1))) # Reshape my output of FC layer so that it's compatible
    hybrid_cnn_lstm_model.add(LSTM(100, dropout=0.6, recurrent_dropout=0.1, input_shape=(100,1), return_sequences=True))

    hybrid_cnn_lstm_model.add(LSTM(70, dropout=0.6, recurrent_dropout=0.1, return_sequences=False))
    # Output layer with Softmax activation 
    hybrid_cnn_lstm_model.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation

    return hybrid_cnn_lstm_model


In [7]:
def get_model_results(model, x_train, y_train, x_valid, y_valid, x_test, y_test): 
    
    # Model parameters
    learning_rate = 2e-3
    epochs = 50
    optimizer = keras.optimizers.Adam(learning_rate)
    
    # Compiling the model
    model.compile(loss='categorical_crossentropy',
         optimizer=optimizer,
         metrics=['accuracy'])

    # Training and validating the model
    model_results = model.fit(x_train,
             y_train,
             batch_size=200,
             epochs=epochs,
             validation_data=(x_valid, y_valid), verbose=True)

    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]
            

# Function to Calculate Accuracies by Subject

In [8]:
def train_by_subjects_random_sampling(subject_ids=[]):
    X_test,y_test,person_train_valid,X_train_valid,y_train_valid,person_test = load_data()

    if subject_ids == []:
        y_train, y_valid, y_test, x_train, x_valid, x_test = preprocessing(y_train_valid, y_test, X_train_valid, X_test)
        basic_cnn_model = generate_basic_cnn_model()  
        return get_model_results(basic_cnn_model, x_train, y_train, x_valid, y_valid, x_test, y_test)

    test_accuracies = {}
    for i in subject_ids:
        X_test,y_test,person_train_valid,X_train_valid,y_train_valid,person_test = load_data()

        X_train_valid = X_train_valid[np.where(person_train_valid==i)[0],:,:]
        y_train_valid = y_train_valid[np.where(person_train_valid==i)[0]]
        X_test = X_test[np.where(person_test==i)[0],:,:]
        y_test = y_test[np.where(person_test==i)[0]]

        y_train, y_valid, y_test, x_train, x_valid, x_test = preprocessing(y_train_valid, y_test, X_train_valid, X_test)
        basic_cnn_model = generate_basic_cnn_model()  
        test_accuracies[i] = get_model_results(basic_cnn_model, x_train, y_train, x_valid, y_valid, x_test, y_test)
    return test_accuracies



In [9]:
train_by_subjects_random_sampling([0, 1, 2, 3, 4, 5, 6, 7, 8])

Epoch 1/50
4/4 [==============================] - 4s 603ms/step - loss: 2.2409 - accuracy: 0.2769 - val_loss: 14.0078 - val_accuracy: 0.2917
Epoch 2/50
4/4 [==============================] - 1s 352ms/step - loss: 1.9177 - accuracy: 0.3590 - val_loss: 16.0199 - val_accuracy: 0.2381
Epoch 3/50
4/4 [==============================] - 1s 382ms/step - loss: 1.6820 - accuracy: 0.4231 - val_loss: 8.7551 - val_accuracy: 0.3810
Epoch 4/50
4/4 [==============================] - 2s 459ms/step - loss: 1.4878 - accuracy: 0.4577 - val_loss: 8.2427 - val_accuracy: 0.2917
Epoch 5/50
4/4 [==============================] - 2s 386ms/step - loss: 1.3962 - accuracy: 0.4679 - val_loss: 12.8948 - val_accuracy: 0.2321
Epoch 6/50
4/4 [==============================] - 2s 416ms/step - loss: 1.2397 - accuracy: 0.5487 - val_loss: 13.9557 - val_accuracy: 0.2321
Epoch 7/50
4/4 [==============================] - 2s 385ms/step - loss: 1.0622 - accuracy: 0.6038 - val_loss: 9.7823 - val_accuracy: 0.2500
Epoch 8/50
4/4 [

KeyboardInterrupt: 